# Matrix Factorization

In [14]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit 
        self.W = np.random.randn(K, self.n_users) if Winit is None else Winit 
        self.n_ratings = Y_data.shape[0] # number of rating 
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        
        
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items
        self.mu = np.zeros(n_objects)
#         self.Y_data_n = self.Y_raw_data
#         return
        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in xrange(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in xrange(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids, ratings = self.Y_data_n[ids, 1].astype(np.int32), self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids, ratings = self.Y_data_n[ids, 0].astype(np.int32), self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in xrange(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in xrange(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in xrange(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print 'iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in xrange(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.as_matrix()


rs = MF(Y_data, K = 2, max_iter = 1000, print_every = 1000)

rs.fit()
rs.pred(6, 1)

iter = 1000 , loss = 0.788308925335 , RMSE train = 0.932907152089


1

In [15]:
print rs.X.dot(rs.W) #+ rs.mu

[[ 3.88091517  3.90291988  1.57922431  0.34704954  1.06643902  1.41712073
   1.3577914 ]
 [ 2.70991132  2.7290018   1.09300399  0.231278    0.73492897  0.99026875
   0.9343738 ]
 [ 2.53060759  2.55496074  1.00366673  0.19660999  0.66925875  0.92604555
   0.84850605]
 [ 2.18464025  1.42229101  2.90929043  2.49438484  2.62364045  0.64352918
   3.6188072 ]
 [ 1.5668107   0.77542396  2.72447018  2.51490887  2.5205531   0.41284591
   3.49673152]]


# Áp dụng lên MovieLens 100k

In [16]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [17]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nUser-based MF, RMSE =', RMSE

iter = 10 , loss = 12.2428756571 , RMSE train = 2.74682324634
iter = 20 , loss = 9.2288768737 , RMSE train = 2.64575131106
iter = 30 , loss = 7.93334444355 , RMSE train = 2.64575131106
iter = 40 , loss = 7.34282354528 , RMSE train = 2.64575131106
iter = 50 , loss = 7.07208867238 , RMSE train = 2.64575131106
iter = 60 , loss = 6.94790704196 , RMSE train = 2.64575131106
iter = 70 , loss = 6.8909482414 , RMSE train = 2.64575131106
iter = 80 , loss = 6.86482381181 , RMSE train = 2.64575131106
iter = 90 , loss = 6.85284196498 , RMSE train = 2.64575131106
iter = 100 , loss = 6.84734657908 , RMSE train = 2.64575131106

User-based MF, RMSE = 2.64575131106


In [86]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nItem-based MF, RMSE =', RMSE

iter = 10 , loss = 5.62220007042 , RMSE train = 1.17955224723
iter = 20 , loss = 2.61441163818 , RMSE train = 1.0053425211
iter = 30 , loss = 1.32260853222 , RMSE train = 0.996519951163
iter = 40 , loss = 0.734133734936 , RMSE train = 0.996180142624
iter = 50 , loss = 0.464450464973 , RMSE train = 0.996177222217
iter = 60 , loss = 0.34078695395 , RMSE train = 0.996179903392
iter = 70 , loss = 0.284077352479 , RMSE train = 0.99618066863
iter = 80 , loss = 0.258071283649 , RMSE train = 0.996180850171
iter = 90 , loss = 0.246145326915 , RMSE train = 0.996180891722
iter = 100 , loss = 0.240676277048 , RMSE train = 0.996180901127

Item-based MF, RMSE = 1.04864244933


In [54]:
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nItem-based MF, RMSE =', RMSE

iter = 10 , loss = 1.17651126822 , RMSE train = 1.6930165132
iter = 20 , loss = 1.10979101182 , RMSE train = 1.653174829
iter = 30 , loss = 1.05052963421 , RMSE train = 1.61696628995
iter = 40 , loss = 0.997617918269 , RMSE train = 1.58393999724
iter = 50 , loss = 0.950149094733 , RMSE train = 1.55371546764
iter = 60 , loss = 0.907375557358 , RMSE train = 1.52596943774
iter = 70 , loss = 0.868676093487 , RMSE train = 1.50042553833
iter = 80 , loss = 0.833530789583 , RMSE train = 1.47684613248
iter = 90 , loss = 0.801501610758 , RMSE train = 1.45502580368
iter = 100 , loss = 0.772217224719 , RMSE train = 1.43478611596

Item-based MF, RMSE = 1.54014628929


In [55]:
RMSE = rs.evaluate_RMSE(rate_train)

In [56]:
print RMSE

1.43478611596


# MovieLens 1M

In [77]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ratings = ratings_base.as_matrix()
ratings[:, :2] -= 1

from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print X_train.shape, X_test.shape

rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, max_iter = 10, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nItem-based MF, RMSE =', RMSE